In [1]:
import numpy as np
import time
import h5py
import io, os, sys
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import FloatTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error

In [2]:
rng = np.random.default_rng(42)

In [3]:
def _sample_grid_uniform(num_sample):
    return rng.uniform(0.0, 1.0, (num_sample, 3))

In [4]:
def _remove_out_of_box(sample_in):
    assert sample_in.shape[1] == 3
    idx = (sample_in[:,0] <= 1.0) & (sample_in[:,1] <= 1.0) & (sample_in[:,2] <= 1.0) & \
          (sample_in[:,0] >= 0.0) & (sample_in[:,1] >= 0.0) & (sample_in[:,2] >= 0.0)
    sample_in = sample_in[idx]
    return sample_in

In [5]:
def _sample_grid_base_on_test(test_output, num_sample, sigma):
    cov = sigma * sigma
    res = np.empty([0, 3])

    w = test_output[:,3]
    w = w / np.sum(w)
    sz = test_output.shape[0]
    freq = rng.multinomial(num_sample, w)

    for i in range(sz):
        temp_out = rng.multivariate_normal(np.array([test_output[i][0], test_output[i][1], test_output[i][2]]), np.diag(np.array([cov, cov, cov])), freq[i])
        temp_out = _remove_out_of_box(temp_out)
        res = np.concatenate([res, temp_out], axis=0)

    return res

In [6]:
def _get_value(sample):
    x1 = sample[:,0]
    x2 = sample[:,1]
    x3 = sample[:,2]
    f1 = np.multiply(x1, np.cos(x2 + x3))
    f2 = np.multiply(x2, np.sin(np.multiply(x1, x3) + 4.0))
    f3 = np.multiply(np.cos(np.sqrt(np.square(x3) + np.square(x1))), np.square(np.log(np.square(x1) + np.power(x2, 4))))
    f4 = x1 - np.multiply(x2, np.exp(np.sin(x3) + np.cos(x1)))
    f5 = x3 + np.divide(np.multiply(x1, x2), (np.square(np.log(np.square(x2))) + np.square(np.sin(x1 + np.cos(x3))) + 1.0))
    h1 = f1 + np.divide(f2, f3 + 1.0)
    h2 = f2 + np.sqrt(np.square(f3) + np.square(f4) + np.square(f5))
    h3 = np.divide(f3, np.log(np.square(f1) + np.square(f4)) + 1.0)
    h4 = np.cos(f4) - np.divide(np.sqrt(np.square(f5) + np.square(f1)), 1.0 + np.square(f2))

    output = np.stack((h1, h2, h3, h4), axis=1)
    return output

In [8]:
num_global_test_sample = 10
sample_global_test = _sample_grid_uniform(num_global_test_sample)
print(sample_global_test)

[[0.77899635 0.13455221 0.53606804]
 [0.51422287 0.85757214 0.46279937]
 [0.3850895  0.63956327 0.26646332]
 [0.13976841 0.47787727 0.41688937]
 [0.23256994 0.36751181 0.36639245]
 [0.32749556 0.37946408 0.68574335]
 [0.29687647 0.94885793 0.91634802]
 [0.48091043 0.32836121 0.53543479]
 [0.84856049 0.65258734 0.80439183]
 [0.53272228 0.63291763 0.28815561]]


In [9]:
val_global_test = _get_value(sample_global_test)
print(val_global_test)

[[ 4.97919186e-01  5.18267614e-01  5.64344756e-01  1.45728642e-01]
 [-6.08841029e-01  2.00793370e+00  1.21238382e-02 -1.33320283e+00]
 [-2.20224880e-03  1.59340199e+00  5.64994487e-01 -4.81759556e-01]
 [ 3.53958587e-02  6.16970046e+00  2.90237686e+00 -6.03189683e-01]
 [ 1.31705227e-01  6.07921840e+00  4.75310544e+00  1.61089941e-02]
 [ 7.61252694e-02  3.15514681e+00  1.66349914e+00 -5.96576180e-01]
 [-9.38936911e-01  4.41677973e+00  1.51954341e-03 -1.94962337e-01]
 [ 1.94668485e-01  1.52046925e+00  1.89623824e+00  7.41550925e-02]
 [-5.53190342e-01  1.36628614e+00  1.98453326e-03 -8.87417793e-01]
 [-2.59063630e-02  1.07048216e+00  3.00483331e-01 -2.90089261e-01]]


In [10]:
num_train_sample = 10
sample_train = _sample_grid_uniform(num_train_sample)
print(sample_train)

[[0.73489316 0.20240459 0.69479813]
 [0.86071907 0.13210284 0.61437974]
 [0.09509575 0.72571563 0.08449322]
 [0.93593982 0.13740793 0.95888025]
 [0.80088418 0.593682   0.7826241 ]
 [0.79511484 0.94602706 0.25338335]
 [0.5900759  0.0950492  0.6161657 ]
 [0.1712913  0.56495061 0.57243051]
 [0.46598515 0.52263178 0.76392339]
 [0.79924472 0.49215322 0.59959344]]


In [12]:
val_train = _get_value(sample_train)
print(val_train)

[[ 0.29309483  0.53972968 -0.37203841  0.18672828]
 [ 0.5074372   0.61492943  0.10189595  0.04638716]
 [-0.15126171  2.01333581  0.63850558 -0.55559368]
 [ 0.29310659  0.90029147 -0.02952686 -0.10608273]
 [-0.41889303  1.27107014  0.01584953 -0.74213678]
 [-0.47062885  0.94755495  0.04273242 -0.49674977]
 [ 0.39543891  0.8896069  -1.71832702  0.2227385 ]
 [-0.03274245  3.76912079  1.22812486 -1.25912078]
 [-0.12057285  1.9473327   0.38392207 -1.17110947]
 [-0.07901008  0.69273148  0.069105   -0.05086469]]


In [13]:
num_test_sample = 10
sample_test = _sample_grid_uniform(num_test_sample)
print(sample_test)

[[0.93123624 0.11973359 0.11710357]
 [0.08770901 0.65786329 0.4186083 ]
 [0.77432142 0.67123141 0.33363776]
 [0.89836655 0.76253215 0.27053494]
 [0.36419202 0.31443998 0.15761165]
 [0.14778337 0.93612746 0.43790404]
 [0.38331982 0.72968571 0.55299307]
 [0.93613999 0.78030149 0.47936956]
 [0.37635947 0.98663154 0.71776024]
 [0.95119466 0.11847858 0.85053368]]


In [14]:
val_test = _get_value(sample_test)
print(val_test)

[[ 8.07811527e-01  5.99185420e-01  9.52367678e-03 -1.31446775e-01]
 [-1.07966424e-01  3.06475150e+00  8.38153433e-01 -1.20197763e+00]
 [-1.69229494e-01  6.45698417e-01  2.34854220e-02 -6.77452742e-02]
 [-2.12635292e-01  4.40661751e-01  9.67126527e-03  6.81681719e-02]
 [ 2.68991254e-01  3.30482738e+00  2.15684073e+01  4.94426868e-01]
 [-6.82217770e-01  2.98821313e+00  1.37836004e-02 -1.18064603e+00]
 [-3.02601119e-01  2.23187566e+00  1.84359473e-01 -1.40387542e+00]
 [-4.48971597e-01  7.83059404e-01  1.53420570e-02 -2.85755903e-01]
 [-9.37370456e-01  3.65354136e+00  1.26313468e-03 -7.73004672e-01]
 [ 4.20899470e-01  8.74603376e-01  7.47099325e-03 -1.20971568e-01]]


In [15]:
class FullModel(torch.nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.fc1 = torch.nn.Linear(4, 32)
        self.fc2 = torch.nn.Linear(32, 16)
        self.fc3 = torch.nn.Linear(16, 8)
        self.fc4 = torch.nn.Linear(8, 3)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x